In [ ]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = ""
os.environ['AWS_SECRET_ACCESS_KEY'] = ""
os.environ['AWS_DEFAULT_REGION'] = ""

In [12]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Exp5__XGBoost_With_HPT")

2025/10/29 10:15:48 INFO mlflow.tracking.fluent: Experiment with name 'Exp5__XGBoost_With_HPT' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/RadhikaMaheshwari/Downloads/mlops_sentiment_analysis/mlruns/538505504330506639', creation_time=1761693348588, experiment_id='538505504330506639', last_update_time=1761693348588, lifecycle_stage='active', name='Exp5__XGBoost_With_HPT', tags={}>

In [13]:

import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB

In [14]:
df = pd.read_csv('cleaned_dataset.csv')

df.shape

(36661, 2)

In [15]:
df['category'].value_counts()

category
 1    15770
 0    12643
-1     8248
Name: count, dtype: int64

In [16]:
df['category'].value_counts(normalize=True)

category
 1    0.430157
 0    0.344862
-1    0.224980
Name: proportion, dtype: float64

In [17]:
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

In [18]:
# Define model configurations
# def get_model_configs():
#     """
#         Define different models to test
#     """
#     return
# {
#     'LogisticRegression': LogisticRegression(random_state=42, max_iter =100),
#     'RandomForestClassifier': RandomForestClassifier(random_state=42, n_estimators=100),
#     'XGBoost':XGBClassifier(random_state = 42, eval_metrics = 'mlogloss'),
#     'LightGBM': LGBMClassifier(random_state=42, verbose = -1),
#     'SVM': SVC(random_state=42, probability=True),
#     'NaiveBayes': MultinomialNB(),
#     'KNN': KNeighborsClassifier(n_neighbors=5)
# }

def get_vectorizer_config():
    """
        Define different vectorization strategies
    """
    return{
        'bow_unigram_5000': CountVectorizer(ngram_range=(1, 1), max_features=5000),
        'bow_bigram_5000': CountVectorizer(ngram_range=(1, 2), max_features=5000),
        'bow_unigram_10000': CountVectorizer(ngram_range=(1, 1), max_features=10000),
        'bow_bigram_10000': CountVectorizer(ngram_range=(1, 2), max_features=10000),
        'tfidf_unigram_5000': TfidfVectorizer(ngram_range=(1, 1), max_features=5000),
        'tfidf_bigram_5000': TfidfVectorizer(ngram_range=(1, 2), max_features=5000),
        'tfidf_trigram_5000': TfidfVectorizer(ngram_range=(1, 3), max_features=5000),
        'tfidf_unigram_10000':TfidfVectorizer(ngram_range=(1, 1), max_features=10000),
        'tfidf_bigram_10000':TfidfVectorizer(ngram_range=(1, 2), max_features=10000),
        'tfidf_trigram_10000': TfidfVectorizer(ngram_range=(1, 3), max_features=10000)
    }

def get_imbalance_methods():
    """
        Define imbalance handling techniques that work well with xgboost
    """
    return ['none', 'smote', 'adasyn','undersampling', 'smote_enn']


In [19]:
def get_xgboost_configs():
    '''
        Define different XGBoost configurations to test
    '''
    return{
        'xgb_light': XGBClassifier(
            n_estimators = 100,
            max_depth = 3,
            learning_rate = 0.1,
            random_state = 42,
            eval_metric = 'mlogloss'        
            ),
        'xgb_medium': XGBClassifier(
            n_estimators = 200,
            max_depth = 5,
            learning_rate = 0.1,
            random_state = 42,
            eval_metric = 'mlogloss'
        ),
        'xgb_deep': XGBClassifier(
            n_estimators = 300,
            max_depth =  7,
            learning_rate = 0.1,
            random_state = 42,
            eval_metric = 'mlogloss'
        ),
        'xgb_conservative': XGBClassifier(
            n_estimators = 200,
            max_depth = 4,
            learning_rate = 0.05,
            subsample = 0.8,
            colsample_bytree = 0.8,
            random_state = 42,
            eval_metric = 'mlogloss'
        ),
        'xgb_aggressive': XGBClassifier(
            n_estimators = 400,
            max_depth = 8,
            learning_rate = 0.2,
            subsample = 1.0,
            colsample_bytree = 1.0,
            random_state = 42,
            eval_metric = 'mlogloss'
        )

    }

In [20]:
def run_xgboost_experiment(xgb_name, 
                           xgb_model, 
                           vectorizer_name,
                           vectorizer,
                           imbalance_method):
    """
        Run a single XGBoost experiment with given configuration
    """
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'],
                                                        df['category'],
                                                        test_size=0.2,
                                                        random_state=42,
                                                        stratify=df['category'])
    
    # Vectorize data
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    # Handle class imbalance
    if imbalance_method == 'smote':
        smote = SMOTE(random_state=42)
        X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
    elif imbalance_method == 'adasyn':
        adasyn = ADASYN(random_state=42)
        X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
    elif imbalance_method == 'undersampling':
        rus = RandomUnderSampler(random_state=42)
        X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
    elif imbalance_method == 'smote_enn':
        smote_enn = SMOTEENN(random_state=42)
        X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)
    
    # Train model
    model = xgb_model.fit(X_train_vec, y_train)

    y_pred_train = model.predict(X_train_vec)
    y_pred_test = model.predict(X_test_vec)

    # Calculate metrics
    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    precision_weighted = precision_score(y_test, y_pred_test, average='weighted')
    recall_weighted = recall_score(y_test, y_pred_test, average = 'weighted')
    precision_macro = precision_score(y_test, y_pred_test, average='macro')
    recall_macro = recall_score(y_test, y_pred_test, average='macro')

    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f'XGBoost_{xgb_name}_{vectorizer_name}_{imbalance_method}')
        mlflow.set_tag('model_type', 'XGBoost')
        mlflow.set_tag('xgb_config', xgb_name)
        mlflow.set_tag('vectorizer_type', vectorizer_name)
        mlflow.set_tag('experiment_type', 'xgboost optimization')

        # Log parameters
        mlflow.log_param('xgb_config', xgb_name)
        mlflow.log_param('vectorizer_name', vectorizer_name)
        mlflow.log_param('imbalance_method', imbalance_method)
        mlflow.log_param('ngram_range', str(vectorizer.ngram_range))
        mlflow.log_param('vectorizer_max_features', vectorizer.max_features)

        # Xgboost specific parameters
        mlflow.log_param('n_estimators', xgb_model.n_estimators)
        mlflow.log_param('max_depth', xgb_model.max_depth)
        mlflow.log_param('learning_rate', xgb_model.learning_rate)
        if hasattr(xgb_model, 'subsample'):
            mlflow.log_param('subsample', xgb_model.subsample)
        if hasattr(xgb_model, 'colsample_bytree'):
            mlflow.log_param('colsample_bytree', xgb_model.colsample_bytree)
        
        # Log metrics
        mlflow.log_metric('accuracy_score_train', train_accuracy)
        mlflow.log_metric('accuracy_score_test', test_accuracy)
        mlflow.log_metric('precision_score_weighted', precision_weighted)
        mlflow.log_metric('precision_score_macro', precision_macro)
        mlflow.log_metric('recall_score_weighted', recall_weighted)
        mlflow.log_metric('recall_score_macro', recall_macro)

        classification_rep = classification_report(y_test, y_pred_test, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f'{label}_{metric}', value)
        
        cm = confusion_matrix(y_test, y_pred_test)
        plt.figure(figsize = (10, 8))
        sns.heatmap(cm, annot = True, fmt = 'd', cmap = 'Blues')
        plt.title(f'XGBoost Confusion Matrix: {xgb_name}+{vectorizer_name}+{imbalance_method}.png')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')

        filename = f'XGBoost Confusion Matrix_{xgb_name}+{vectorizer_name}+{imbalance_method}.png'
        plt.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close()

        # Feature importance (top 20)
        if hasattr(xgb_model, 'feature_importances_'):
            feature_names = vectorizer.get_feature_names_out()
            importance_df = pd.DataFrame({
                'feature': feature_names,
                'importance': xgb_model.feature_importances_
            }).sort_values('importance', ascending=False).head(20)

            plt.figure(figsize=(10, 8))
            sns.barplot(data = importance_df,
                        x ='importance',
                        y = 'feature')
            plt.title(f'Top 20 feature importance: {xgb_name}')
            plt.tight_layout()

            importance_filename = f'xgb_feature_importance_{xgb_name}_{vectorizer_name}_{imbalance_method}.png'
            plt.savefig(importance_filename)
            mlflow.log_artifact(importance_filename)
            plt.close()
        
        # Log model
        mlflow.sklearn.log_model(
            xgb_model,
            name = f'xgb_model_{xgb_name}_{vectorizer_name}_{imbalance_method}',
            registered_model_name = f'reg_xgb_model_{xgb_name}_{vectorizer_name}_{imbalance_method}',
        )
        print(f'Completed: {xgb_name}_{vectorizer_name}_{imbalance_method}')
        


In [21]:
def run_all_experiments():
    """
        Run comprehensive XGBoost experiments with all combinations
    """
    xgb_configs = get_xgboost_configs()
    vectorizers = get_vectorizer_config()
    imbalance_methods = get_imbalance_methods()

    results = []
    total_experiments = len(xgb_configs) * len(vectorizers) * len(imbalance_methods)
    current_experiment = 0

    print(f'Starting {total_experiments} XGBoost experiments')

    for xgb_name, xgb_model in xgb_configs.items():
        for vec_name, vectorizer in vectorizers.items():
            for imbalance_method in imbalance_methods:
                current_experiment += 1
                print(f'XGBoost Experiment: {current_experiment}/{total_experiments}')

                try:
                    result = run_xgboost_experiment(xgb_name = xgb_name, 
                           xgb_model = xgb_model, 
                           vectorizer_name = vec_name,
                           vectorizer = vectorizer,
                           imbalance_method = imbalance_method)
                except Exception as e:
                    print(f"Error occurred: {e}")
    

In [22]:
run_all_experiments()

Starting 250 XGBoost experiments
XGBoost Experiment: 1/250
🏃 View run XGBoost_xgb_light_bow_unigram_5000_none at: http://127.0.0.1:5000/#/experiments/538505504330506639/runs/e05b40f7849545e4b9d506e06b56ebcf
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/538505504330506639
Error occurred: API request to endpoint /api/2.0/mlflow/logged-models failed with error code 404 != 200. Response body: '<!doctype html>
<html lang=en>
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>
'
XGBoost Experiment: 2/250
🏃 View run XGBoost_xgb_light_bow_unigram_5000_smote at: http://127.0.0.1:5000/#/experiments/538505504330506639/runs/8d08d66374e74ad1a04b5cfd3ec804ca
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/538505504330506639
Error occurred: API request to endpoint /api/2.0/mlflow/logged-models failed with error code 404 != 200. Response body: '<!doctype htm